<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/ECEG/eceg_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
import random

class Point:
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

    def is_infinity(self):
        """Check if the point is the point at infinity."""
        return self.x is None and self.y is None

    def __eq__(self, other):
        """Custom equality check for Point objects."""
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        return False

    def __repr__(self):
        if self.is_infinity():
            return "Point at Infinity"
        return f"Point({self.x}, {self.y})"

class ECEG:
  def __init__(self):
    self.a = 6
    self.b = 7
    self.p = 61
    self.base_point = self.generate_random_valid_point()

  def elliptic_curve_equation(self, x):
    return (x**3 + self.a*x + self.b) % self.p

  # def is_on_curve(self, x, y):
  #   return self.elliptic_curve_equation(x)  == (y**2) % self.p

  # def generate_random_point(self):
  #   while True:
  #     x = random.randint(1, self.p - 1)
  #     y = random.randint(1, self.p - 1)
  #     if self.is_on_curve(x, y):
  #       return Point(x, y)

  def is_on_curve(self, x, y):
        """Check if a point (x, y) lies on the curve."""
        if x is None or y is None:
            return False
        return (y**2 - (x**3 + self.a * x + self.b)) % self.p == 0

  def generate_random_valid_point(self):
        """Generate a random point that lies on the elliptic curve."""
        while True:
            x = random.randint(0, self.p - 1)  # Random x-coordinate
            y_squared = (x**3 + self.a * x + self.b) % self.p  # Compute y^2

            # Check if y_squared is a quadratic residue modulo p
            if pow(y_squared, (self.p - 1) // 2, self.p) == 1:
                # Find a valid y-coordinate
                for y in range(self.p):
                    if (y**2) % self.p == y_squared:
                        return Point(x, y)

  # def calc_point_add(self, P, Q):
  #   R = Point()  # Initialize the result point R
  #   print(P, Q)

  #   slope = ((Q.y - P.y) / (Q.x - P.x))

  #   # Calculate Rx
  #   R.x = (slope**2 - P.x - Q.x)

  #   # Calculate Ry
  #   R.y = (slope * (P.x - R.x) - P.y)

  #   return R

  def calc_point_add(self, P, Q):
    """Calculate the addition of two points P and Q on the elliptic curve."""
    R = Point()  # Initialize the result point R

    if P.is_infinity():
        return Q
    if Q.is_infinity():
        return P

    # Handle the case where P and Q are inverses
    if P.x == Q.x and (P.y != Q.y or P.y == 0):
        return Point()  # Point at infinity

    # Calculate slope
    if P.x == Q.x and P.y == Q.y:
        # Point doubling
        slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p
    else:
        # Regular point addition
        slope = (Q.y - P.y) * pow(Q.x - P.x, -1, self.p) % self.p

    # Calculate Rx
    R.x = (slope**2 - P.x - Q.x) % self.p

    # Calculate Ry
    R.y = (slope * (P.x - R.x) - P.y) % self.p

    return R

  def calc_point_doubling(self, P):
      """Calculate the point doubling 2P = P + P on the elliptic curve."""
      R = Point()  # Initialize the result point R

      if P.is_infinity() or P.y == 0:
          # Point at infinity for vertical tangent or zero y-coordinate
          return Point()

      # Calculate slope for point doubling
      slope = (3 * P.x**2 + self.a) * pow(2 * P.y, -1, self.p) % self.p

      # Calculate Rx
      R.x = (slope**2 - 2 * P.x) % self.p

      # Calculate Ry
      R.y = (slope * (P.x - R.x) - P.y) % self.p

      return R

  # def calc_point_subtraction(self, P, Q):
  #     """Calculate the subtraction of two points P - Q on the elliptic curve."""
  #     # Negate Q to get -Q
  #     Q_neg = Point(Q.x, (-Q.y) % self.p)

  #     # Add P and -Q
  #     return self.calc_point_add(P, Q_neg)

  def calc_point_subtraction(self, P, Q):
    """Calculate the subtraction of two points P - Q on the elliptic curve."""
    # Check if Q is the point at infinity
    if Q.is_infinity():
        return P  # P - Q = P when Q is the point at infinity

    # Check if P is the point at infinity
    if P.is_infinity():
        Q_neg = Point(Q.x, (-Q.y) % self.p)  # Negate Q
        return Q_neg  # 0 - Q = -Q

    # Negate Q to get -Q
    Q_neg = Point(Q.x, (-Q.y) % self.p)

    # Add P and -Q
    return self.calc_point_add(P, Q_neg)


  def calc_point_multiplication(self, P, k):
    """Calculate kP using the double-and-add method."""
    R = Point()  # Start with the point at infinity
    current_point = P

    while k > 0:
        if k % 2 == 1:
            # If the current bit is 1, add the current point
            R = self.calc_point_add(R, current_point)
        # Double the point
        current_point = self.calc_point_add(current_point, current_point)
        k //= 2  # Move to the next bit

    return R

  def generate_keys(self):
        """Generate a private and public key pair."""
        # Private key d: Random scalar
        private_key = random.randint(1, self.p - 1)

        # Public key e2 = d * e1 (base point)
        public_key = self.calc_point_multiplication(self.base_point, private_key)

        return private_key, public_key

  def encrypt(self, plaintext_point, public_key):
        """
        Encrypt a point on the elliptic curve using the public key.

        Args:
            plaintext_point (Point): The plaintext point to encrypt (P).
            public_key (Point): The public key (e2).

        Returns:
            tuple: A tuple containing the cipher points (C1, C2).
        """
        # Generate a random ephemeral key k
        k = random.randint(1, self.p - 1)

        # Calculate C1 = k * e1 (base point)
        C1 = self.calc_point_multiplication(self.base_point, k)

        # Calculate k * e2 (public key)
        k_e2 = self.calc_point_multiplication(public_key, k)

        # Calculate C2 = P + k * e2
        C2 = self.calc_point_add(plaintext_point, k_e2)

        return C1, C2

  def decrypt(self, C1, C2, private_key):
        """
        Decrypt a ciphertext pair (C1, C2) using the private key.

        Args:
            C1 (Point): The first ciphertext point.
            C2 (Point): The second ciphertext point.
            private_key (int): The private key (d).

        Returns:
            Point: The plaintext point (P).
        """
        # Calculate d * C1
        d_C1 = self.calc_point_multiplication(C1, private_key)

        # Subtract d * C1 from C2 to get the plaintext point
        plaintext_point = self.calc_point_subtraction(C2, d_C1)

        return plaintext_point

In [227]:
import unittest

class TestECEG(unittest.TestCase):
    def setUp(self):
        # Initialize the elliptic curve
        self.elliptic = ECEG()

        # Generate keys
        self.private_key, self.public_key = self.elliptic.generate_keys()

        # Generate a plaintext point for encryption
        self.plaintext_point = self.elliptic.generate_random_valid_point()

    def test_point_on_curve(self):
        """Test if a randomly generated point lies on the elliptic curve."""
        # Verify that the generated random point satisfies the curve equation
        self.assertTrue(
            self.elliptic.is_on_curve(self.elliptic.base_point.x, self.elliptic.base_point.y),
            f"Point is not on the curve. Point: {self.elliptic.base_point}"
        )

    def test_addition_doubling(self):
        """Test if P + P equals 2P."""
        # Perform addition: P + P
        result_add = self.elliptic.calc_point_add(self.elliptic.base_point, self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_add.x, result_add.y),
            f"Addition result is not on the curve. Result: {result_add}"
        )

        # Perform doubling: 2P
        result_doubling = self.elliptic.calc_point_doubling(self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_doubling.x, result_doubling.y),
            f"Doubling result is not on the curve. Result: {result_doubling}"
        )

        # Verify that P + P equals 2P
        self.assertEqual(
            result_add,
            result_doubling,
            f"P + P does not equal 2P. Addition Result: {result_add}, Doubling Result: {result_doubling}"
        )

    def test_calc_addition_subtraction(self):
        """Test if (P + P) - P equals P."""
        # Perform addition: P + P
        result_add = self.elliptic.calc_point_add(self.elliptic.base_point, self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_add.x, result_add.y),
            f"Addition result is not on the curve. Result: {result_add}"
        )

        # Perform subtraction: (P + P) - P
        result_subtraction = self.elliptic.calc_point_subtraction(result_add, self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_subtraction.x, result_subtraction.y),
            f"Subtraction result is not on the curve. Result: {result_subtraction}"
        )

        # Verify that (P + P) - P equals P
        self.assertEqual(
            result_subtraction,
            self.elliptic.base_point,
            f"(P + P) - P does not equal P. Subtraction Result: {result_subtraction}, Random Point: {self.elliptic.base_point}"
        )

    def test_calc_doubling_multiplication(self):
        """Test if 2P doubling 2P equals 4P."""
        # Perform first doubling: 2P
        result_doubling_first = self.elliptic.calc_point_doubling(self.elliptic.base_point)
        self.assertTrue(
            self.elliptic.is_on_curve(result_doubling_first.x, result_doubling_first.y),
            f"Doubling result first is not on the curve. Result: {result_doubling_first}"
        )

        # Perform second doubling: 4P
        result_doubling_second = self.elliptic.calc_point_doubling(result_doubling_first)
        self.assertTrue(
            self.elliptic.is_on_curve(result_doubling_second.x, result_doubling_second.y),
            f"Doubling result second is not on the curve. Result: {result_doubling_second} (result_doubling_first={result_doubling_first} )"
        )

        # Calculate 4P using scalar multiplication
        k = 4
        result_multiplication = self.elliptic.calc_point_multiplication(self.elliptic.base_point, k)
        self.assertTrue(
            self.elliptic.is_on_curve(result_multiplication.x, result_multiplication.y),
            f"Multiplication result is not on the curve. Result: {result_multiplication}"
        )

        # Verify that 2P doubling 2P equals 4P
        self.assertEqual(
            result_doubling_second,
            result_multiplication,
            f"2P doubling 2P does not equal 4P. Doubling Result: {result_doubling_second}, Multiplication Result: {result_multiplication}"
        )

    def test_generate_keys(self):
        """Test if the key generation produces a valid private-public key pair."""
        # Generate keys
        private_key, public_key = self.elliptic.generate_keys()

        # Ensure private key is within valid range
        self.assertGreater(
            private_key, 0,
            f"Private key must be greater than 0. Private Key: {private_key}"
        )
        self.assertLess(
            private_key, self.elliptic.p,
            f"Private key must be less than {self.elliptic.p}. Private Key: {private_key}"
        )

        # Ensure public key is on the curve
        self.assertTrue(
            self.elliptic.is_on_curve(public_key.x, public_key.y),
            f"Public key does not lie on the curve. Public Key: {public_key} (private_key={private_key}, base_point={self.elliptic.base_point})"
        )

        # Ensure public key is calculated as d * e1
        expected_public_key = self.elliptic.calc_point_multiplication(self.elliptic.base_point, private_key)
        self.assertEqual(
            public_key,
            expected_public_key,
            f"Public key is not correctly calculated as d * e1. Expected: {expected_public_key}, Actual: {public_key}"
        )

    def test_encrypt(self):
        """Test if the encryption function generates valid cipher points."""
        # Encrypt the plaintext point
        C1, C2 = self.elliptic.encrypt(self.plaintext_point, self.public_key)

        # Ensure C1 lies on the curve
        self.assertTrue(
            self.elliptic.is_on_curve(C1.x, C1.y),
            f"C1 does not lie on the curve. C1: {C1}"
        )

        # Ensure C2 lies on the curve
        self.assertTrue(
            self.elliptic.is_on_curve(C2.x, C2.y),
            f"C2 does not lie on the curve. C2: {C2}"
        )

    def test_decrypt(self):
        """Test if decryption recovers the original plaintext."""
        # Encrypt the plaintext point
        C1, C2 = self.elliptic.encrypt(self.plaintext_point, self.public_key)

        # Decrypt the ciphertext
        decrypted_point = self.elliptic.decrypt(C1, C2, self.private_key)

        # Verify that decryption recovers the original plaintext
        self.assertEqual(
            decrypted_point,
            self.plaintext_point,
            f"Decryption failed. Decrypted Point: {decrypted_point}, Original Point: {self.plaintext_point}"
        )

        print(f"Decryption passed. Decrypted Point: {decrypted_point}, Original Point: {self.plaintext_point} with cipher {C1, C2}")

# Run the unittest tests
if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


test_addition_doubling (__main__.TestECEG)
Test if P + P equals 2P. ... ok
test_calc_addition_subtraction (__main__.TestECEG)
Test if (P + P) - P equals P. ... ok
test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P. ... ok
test_decrypt (__main__.TestECEG)
Test if decryption recovers the original plaintext. ... ok
test_encrypt (__main__.TestECEG)
Test if the encryption function generates valid cipher points. ... ok
test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair. ... ok
test_point_on_curve (__main__.TestECEG)
Test if a randomly generated point lies on the elliptic curve. ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.021s

OK


Addition and doubling test passed. P + P = 2P. Result: Point(40, 21)
Addition and subtraction test passed. (P + P) - P = P. Result: Point(34, 29)
Doubling and multiplication test passed. 2P doubling 2P = 4P. Result: Point(60, 0)
Decryption test passed. Decrypted Point: Point(13, 5), Original Point: Point(13, 5) with cipher (Point(28, 44), Point(6, 25))
Encryption test passed. Cipher Points: C1=Point(60, 0), C2=Point(4, 20)
Key generation test passed. Private Key: 28, Public Key: Point(57, 38)
Point on curve test passed. Point: Point(54, 7)


# Dump code

In [218]:
"""
error list:

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-155-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=16, base_point=Point(53, 22))
-

-
FAIL: test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-170-0a96d71a6351>", line 76, in test_calc_doubling_multiplication
    self.assertTrue(
AssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(21, 0)
-

-
FAIL: test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-175-0a96d71a6351>", line 76, in test_calc_doubling_multiplication
    self.assertTrue(
AssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(41, 0) )
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-175-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=56, base_point=Point(54, 7))
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-180-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=56, base_point=Point(32, 24))
-

-
FAIL: test_calc_doubling_multiplication (__main__.TestECEG)
Test if 2P doubling 2P equals 4P.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-187-0a96d71a6351>", line 76, in test_calc_doubling_multiplication
    self.assertTrue(
AssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(60, 0) )
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-196-0a96d71a6351>", line 112, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=48, base_point=Point(50, 14))
-

-
FAIL: test_generate_keys (__main__.TestECEG)
Test if the key generation produces a valid private-public key pair.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-211-802f94a9058a>", line 115, in test_generate_keys
    self.assertTrue(
AssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=16, base_point=Point(17, 9))
-

-
ERROR: test_decrypt (__main__.TestECEG)
Test if decryption recovers the original plaintext.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-221-16ec17c776f0>", line 151, in test_decrypt
    decrypted_point = self.elliptic.decrypt(C1, C2, self.private_key)
  File "<ipython-input-216-cd268fb89798>", line 198, in decrypt
    plaintext_point = self.calc_point_subtraction(C2, d_C1)
  File "<ipython-input-216-cd268fb89798>", line 127, in calc_point_subtraction
    Q_neg = Point(Q.x, (-Q.y) % self.p)
TypeError: bad operand type for unary -: 'NoneType'
-



"""

' \nerror list:\n\n-\nFAIL: test_generate_keys (__main__.TestECEG)\nTest if the key generation produces a valid private-public key pair.\n----------------------------------------------------------------------\nTraceback (most recent call last):\n  File "<ipython-input-155-0a96d71a6351>", line 112, in test_generate_keys\n    self.assertTrue(\nAssertionError: False is not true : Public key does not lie on the curve. Public Key: Point at Infinity (private_key=16, base_point=Point(53, 22))\n-\n\n-\nFAIL: test_calc_doubling_multiplication (__main__.TestECEG)\nTest if 2P doubling 2P equals 4P.\n----------------------------------------------------------------------\nTraceback (most recent call last):\n  File "<ipython-input-170-0a96d71a6351>", line 76, in test_calc_doubling_multiplication\n    self.assertTrue(\nAssertionError: False is not true : Doubling result second is not on the curve. Result: Point at Infinity (result_doubling_first=Point(21, 0) \n-\n\n-\nFAIL: test_calc_doubling_multipl

## test case

In [219]:
def test_calc_add_and_doubling():
    elliptic = ECEG()

    # Generate a valid random point
    random_point = elliptic.generate_random_point()
    print(f"Random Point: {random_point}")

    # Ensure the generated point lies on the curve
    assert elliptic.is_on_curve(random_point.x, random_point.y), "Generated point is not on the curve."

    # Perform point addition (P + P)
    result_add = elliptic.calc_point_add(random_point, random_point)
    print(f"Result of P + P (Addition): {result_add}")

    # Perform point doubling (2P)
    result_doubling = elliptic.calc_point_doubling(random_point)
    print(f"Result of 2P (Doubling): {result_doubling}")

    # Ensure both addition and doubling results lie on the curve
    assert elliptic.is_on_curve(result_add.x, result_add.y), "Addition result is not on the curve."
    assert elliptic.is_on_curve(result_doubling.x, result_doubling.y), "Doubling result is not on the curve."

    if (result_add == result_doubling):
      print("Test passed: Point addition and doubling produce the same result.")
    else:
      Exception("Test failed: Point addition and doubling do not produce the same result.")

# Run the test
test_calc_add_and_doubling()

AttributeError: 'ECEG' object has no attribute 'generate_random_point'

In [ ]:
def test_calc_subtraction():
  elliptic = ECEG()
  random_point = elliptic.generate_random_point()
  print(random_point)

  result_add = elliptic.calc_point_add(random_point, random_point)
  print(result_add)

  result_subtraction = elliptic.calc_point_subtraction(result_add, random_point)
  print(result_subtraction)

  assert result_subtraction != random_point, "should be same"

test_calc_subtraction()

In [ ]:
def test_calc_multiplication():
  elliptic = ECEG()
  random_point = elliptic.generate_random_point()
  print(random_point)

  result_doubling = elliptic.calc_point_doubling(random_point)
  print(result_doubling)

  result_doubling_2 = elliptic.calc_point_doubling(result_doubling)
  print(result_doubling_2)

  result_multiplication = elliptic.calc_point_multiplication(random_point, 2)
  print(result_multiplication)

  assert result_multiplication != result_doubling_2, "should be same"

test_calc_multiplication()

In [ ]:
5 * 2 * 2, 5 * 2